In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import convert_and_clean, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP"]
init_funds = {"BTC":'0.00000000', "ETH":'0.00000000', "LTC":'0.00000000', "XRP":'0.00000000', "USDT":'100.00000000'}

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=150)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [3]:
# Training params
nb_steps = 200
batch_size = 4
nb_max_episode_steps = 7

agent = apriori.MomentumTrader()
opt_params, info = agent.fit(env, nb_steps, batch_size, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 200 steps with batch size 4...
Optimization step 11/200, step reward: -0.023483500587885873, ETC: 0 days 00:14:21.074401                     
Keyboard Interrupt: Stoping backtest
Elapsed steps: 6/7, 85 % done.

Optimization interrupted by user.

 None 
 {'OOD': False, 'Error': False, 'ValueError': False, 'ActionError': False}
>> step 98/2099, 4 % done, Cumulative Reward: -0.00014129701879496935, ETC: 0 days 00:04:19.363091  
Keyboard Interrupt: Stoping backtest
Elapsed steps: 98/2400, 4 % done.

################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.251293
Portifolio / Benchmark Sharpe ratio:      -3.213286 / 0.977577
Portifolio / Benchmark Omega ratio:       0.514104 / 1.168544
Portifolio / Benchmark max drawdown:      -1.000000 / -0.999941


In [4]:
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))

agent.test(env, verbose=True)
env.plot_results();

>> step 18/4497, 0 % done, Cumulative Reward: -0.02871277727999393, ETC: 0 days 00:09:46.294272    
Keyboard Interrupt: Stoping backtest
Elapsed steps: 18/4798, 0 % done.

################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.104062
Portifolio / Benchmark Sharpe ratio:      -6.891370 / -6.537998
Portifolio / Benchmark Omega ratio:       0.256540 / 0.286490
Portifolio / Benchmark max drawdown:      -1.002762 / -1.007084
